

---

### **Objetivo:**
- Capturar os dados do **IPCA - Variação Acumulada no Ano (%)** e **IPCA - Variação Mensal (%)** a partir da página do IBGE, usando o **Selenium**.

---

### **1. Criar a pasta e o arquivo .ipynb**
Crie uma pasta no seu diretório de trabalho e dentro dela crie um arquivo Jupyter Notebook (com extensão `.ipynb`).

---

### **2. Instalar a extensão Jupyter**
Instale a extensão Jupyter em seu editor de código para usar notebooks interativos. Use o seguinte comando no terminal:

```bash
pip install notebook
```

---

### **3. Criar o Ambiente Virtual**
No terminal, navegue até o diretório do seu projeto e crie um ambiente virtual:

```bash
python -m venv venv
```

Para garantir que você está usando a versão mais recente do `pip`, execute:

```bash
python.exe -m pip install --upgrade pip
```

Depois, ative o ambiente virtual com o comando:

```bash
venv\Scripts\activate  # Windows
source venv/bin/activate  # Linux/Mac
```

---

### **4. Instalar as Dependências**
Instale as bibliotecas necessárias com os seguintes comandos:

```bash
pip install selenium
pip install webdriver-manager
pip install pandas
```

Para salvar as dependências em um arquivo `requirements.txt`, execute:

```bash
pip freeze > requirements.txt
```

Caso precise instalar as dependências em outro ambiente, basta rodar:

```bash
pip install -r requirements.txt
```

---

### **5. Verificar a versão do Chrome e instalar o ChromeDriver**
Acesse `chrome://settings/help` para verificar a versão do Chrome (exemplo: 132.0.6834.111). Se for necessário, atualize seu Chrome.

Depois, baixe o **ChromeDriver** compatível com sua versão de Chrome em [Chrome for Testing](https://googlechromelabs.github.io/chrome-for-testing/#stable).

Coloque o ChromeDriver no diretório `C:\tools` ou outro de sua preferência.

---

### **6. Adicionar o caminho do ChromeDriver no PATH**
1. Acesse as **Variáveis de Ambiente**:
   - Pressione `Win + S` e digite "Editar variáveis de ambiente do sistema".
   - Clique em **Variáveis de Ambiente**.
   
2. Na janela que abrir, edite a variável **Path**:
   - Selecione **Path** e clique em **Editar**.
   - Adicione o caminho do ChromeDriver: `C:\tools\` (não é necessário colocar `chromedriver.exe` no Path).

3. Salve as alterações.

4. Para garantir que a variável foi corretamente configurada, reinicie o Prompt de Comando e digite:

```bash
echo %PATH%
```

Verifique se o caminho do ChromeDriver está correto.

---

### **7. Iniciar o ChromeDriver**
No terminal, execute o seguinte comando para iniciar o ChromeDriver:

```bash
C:\tools\chromedriver.exe
```

Isso fará o ChromeDriver ficar em execução.

---

### **8. Criar o script para automatizar o processo com Selenium**
Crie um script Python para acessar a página, interagir com os botões e capturar os dados. Aqui está o código completo com as modificações solicitadas:

```python
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

def esperar_elemento(driver, by, value, tempo=10):
    """Aguarda até que um elemento esteja presente na página."""
    return WebDriverWait(driver, tempo).until(
        EC.presence_of_element_located((by, value))
    )

def esperar_e_clicar(driver, xpath, descricao, tempo=30, tentativas=3):
    """Aguarda e clica em um elemento específico."""
    for tentativa in range(tentativas):
        try:
            print(f"Aguardando {descricao}... (Tentativa {tentativa + 1})")
            elemento = esperar_elemento(driver, By.XPATH, xpath, tempo)
            driver.execute_script("arguments[0].scrollIntoView(true);", elemento)
            time.sleep(2)
            driver.execute_script("arguments[0].click();", elemento)
            time.sleep(2)
            return True
        except Exception as e:
            if tentativa == tentativas - 1:
                print(f"Erro final ao clicar em {descricao}: {e}")
                return False
            time.sleep(2)

def capturar_dados(driver):
    """Captura os dados de IPCA e imprime em uma lista de DataFrame."""
    try:
        # Captura dados da Variação Acumulada no Ano (%)
        acumulado_xpath = "//td[@class='x-col-0' and @title='%']"
        acumulado_elemento = driver.find_elements(By.XPATH, acumulado_xpath)
        acumulado_valores = [elemento.text for elemento in acumulado_elemento]

        # Captura dados da Variação Mensal (%)
        mensal_xpath = "//tr//td[@class='x-col-0' and @title='%']"
        mensal_elemento = driver.find_elements(By.XPATH, mensal_xpath)
        mensal_valores = [elemento.text for elemento in mensal_elemento]

        print("\nValores encontrados:")
        print("Variação Acumulada no Ano (%):", acumulado_valores)
        print("Variação Mensal (%):", mensal_valores)

        # Criar DataFrame
        dados = {
            "Variação Acumulada no Ano (%)": acumulado_valores,
            "Variação Mensal (%)": mensal_valores
        }

        df = pd.DataFrame(dados)
        print("\nDataFrame criado:")
        print(df)

        return df
    except Exception as e:
        print(f"Erro ao capturar dados: {e}")
        raise

# Configurações do ChromeDriver
options = Options()
driver = webdriver.Chrome(options=options)

try:
    # Abrir URL
    url = "https://sidra.ibge.gov.br/tabela/7060"
    driver.get(url)
    time.sleep(7)

    # Clicar no botão "Marcar todos"
    if not esperar_e_clicar(driver, "//button[@data-cmd='marcarTudo']", "Marcar todos"):
        raise Exception("Falha ao marcar todos os elementos.")

    # Clicar no botão "Visualizar"
    if not esperar_e_clicar(driver, "//button[contains(text(),'Visualizar')]", "Visualizar"):
        raise Exception("Falha ao clicar em 'Visualizar'.")

    print("Aguardando resultados carregarem...")
    time.sleep(10)

    # Capturar dados e imprimir DataFrame
    df_dados = capturar_dados(driver)

except Exception as e:
    print(f"Erro durante a execução: {e}")
finally:
    driver.quit()
```

---

### **9. Explicação do Código:**
1. **Aguarda os elementos da página** - O script aguarda os elementos "Marcar todos" e "Visualizar" para interagir com eles.
2. **Clica nos botões** - Clica no botão "Marcar todos" para selecionar os elementos e no botão "Visualizar" para carregar os resultados.
3. **Captura os dados de IPCA** - Usa o XPath para pegar os valores de Variação Acumulada no Ano (%) e Variação Mensal (%).
4. **Cria um DataFrame** - Armazena os dados em um DataFrame do `pandas` para organização e análise.

---

### **10. Resultados Esperados:**
- O script imprimirá os dados coletados no console.
- Será exibido um DataFrame com as variações mensais e acumuladas no ano do IPCA.

Agora você pode rodar esse script para capturar os dados de forma automatizada. Se precisar de mais alguma coisa, me avise!